In [1]:
# ============================================================
# Install All Dependencies (Clean + Correct Order)
# ============================================================

# 1. KaggleHub (for dataset download)
!pip install kagglehub --quiet

# 2. PyTorch 2.2.0 with CUDA 11.8  (required for pyg-lib and PyG wheels)
!pip install torch==2.2.0+cu118 torchvision==0.17.0+cu118 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu118 --quiet

# 3. pyg-lib (needed for NeighborLoader)
!pip install pyg-lib -f https://data.pyg.org/whl/torch-2.2.0+cu118.html --quiet

# 4. PyTorch Geometric (must be installed AFTER torch + pyg-lib)
!pip install torch-geometric --quiet

# Install Gradio
!pip install -q gradio

print("✔ All packages installed successfully.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.6/811.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 109.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 141.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# ============================================================
# Section 0 — Imports & Global Config
# ============================================================

import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    precision_recall_curve
)

from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv

# Device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print("Torch version:", torch.__version__)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-package

Using device: cuda
Torch version: 2.2.0+cu118


In [3]:
# ============================================================
# Section 1 — Load PaySim Dataset using KaggleHub
# ============================================================

import kagglehub

# Download PaySim dataset
dataset_path = kagglehub.dataset_download("ealaxi/paysim1")

# Build the CSV path dynamically
csv_path = os.path.join(dataset_path, "PS_20174392719_1491204439457_log.csv")

# Load into DataFrame
df = pd.read_csv(csv_path, low_memory=False)

print("CSV loaded from:", csv_path)
print("Initial rows:", len(df))

# Keep only columns needed
df = df[[
    "step", "type", "amount",
    "oldbalanceOrg", "newbalanceOrig",
    "oldbalanceDest", "newbalanceDest",
    "nameOrig", "nameDest",
    "isFraud"
]].copy()

print("Rows after column selection:", len(df))
print(df.head())
print("\nFraud distribution:")
print(df["isFraud"].value_counts())
print("Fraud ratio:", df["isFraud"].mean())



Using Colab cache for faster access to the 'paysim1' dataset.
CSV loaded from: /kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv
Initial rows: 6362620
Rows after column selection: 6362620
   step      type    amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  \
0     1   PAYMENT   9839.64       170136.0       160296.36             0.0   
1     1   PAYMENT   1864.28        21249.0        19384.72             0.0   
2     1  TRANSFER    181.00          181.0            0.00             0.0   
3     1  CASH_OUT    181.00          181.0            0.00         21182.0   
4     1   PAYMENT  11668.14        41554.0        29885.86             0.0   

   newbalanceDest     nameOrig     nameDest  isFraud  
0             0.0  C1231006815  M1979787155        0  
1             0.0  C1666544295  M2044282225        0  
2             0.0  C1305486145   C553264065        1  
3             0.0   C840083671    C38997010        1  
4             0.0  C2048537720  M1230701703        0  

Fra

In [4]:
# ============================================================
# Section 2 — Train/Val/Test Split
# ============================================================

# We will use these columns as features for GAN + GNN
feature_cols = [
    "step", "type", "amount",
    "oldbalanceOrg", "newbalanceOrig",
    "oldbalanceDest", "newbalanceDest"
]

X = df[feature_cols].copy()
y = df["isFraud"].astype(int).values

# First split: train (64%), temp (36%)
X_train, X_temp, y_train, y_temp, df_train, df_temp = train_test_split(
    X, y, df, test_size=0.36, random_state=42, stratify=y
)

# Second split: temp → validation (20%), test (16%)
X_val, X_test, y_val, y_test, df_val, df_test = train_test_split(
    X_temp, y_temp, df_temp, test_size=(16/36), random_state=42, stratify=y_temp
)

print("Train size:", len(X_train))
print("Validation size:", len(X_val))
print("Test size:", len(X_test))

print("\nFraud ratio per split:")
print("Train:", y_train.mean())
print("Val:", y_val.mean())
print("Test:", y_test.mean())


Train size: 4072076
Validation size: 1272524
Test size: 1018020

Fraud ratio per split:
Train: 0.001290742117779727
Val: 0.0012911347840983745
Test: 0.001290740849885071


In [5]:
# ============================================================
# Section 3 — Prepare Fraud Data for GAN Training
# ============================================================

print("Real fraud samples in TRAIN:", y_train.sum())

# Select fraud rows from TRAIN split
fraud_mask = (y_train == 1)
X_fraud = X_train[fraud_mask]

print("Fraud shape:", X_fraud.shape)
display(X_fraud.head())

# -----------------------------
# 1. One-hot encode "type"
# -----------------------------
enc = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
enc.fit(X_train[["type"]])  # fit on training set

type_fraud_oh = enc.transform(X_fraud[["type"]])

# -----------------------------
# 2. Normalize numeric features
# -----------------------------
num_cols = ["step", "amount", "oldbalanceOrg", "newbalanceOrig", "oldbalanceDest", "newbalanceDest"]

scaler = StandardScaler()
scaler.fit(X_train[num_cols])   # fit only on TRAIN numerics

num_fraud_scaled = scaler.transform(X_fraud[num_cols])

# -----------------------------
# 3. Final fraud feature matrix for GAN
# -----------------------------
fraud_features = np.hstack([num_fraud_scaled, type_fraud_oh]).astype(np.float32)

print("Final fraud GAN feature matrix shape:", fraud_features.shape)

Real fraud samples in TRAIN: 5256
Fraud shape: (5256, 7)


,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
3956497,288,CASH_OUT,1905476.22,1905476.22,0.0,0.00,1905476.22
3960330,296,TRANSFER,5591414.49,5591414.49,0.0,0.00,0.00
1059575,105,CASH_OUT,188854.39,188854.39,0.0,0.00,188854.39
4785627,339,CASH_OUT,164382.72,164382.72,0.0,445530.63,609913.36
1511333,149,CASH_OUT,3567795.25,3567795.25,0.0,0.00,3567795.25


Final fraud GAN feature matrix shape: (5256, 11)


In [6]:
# ============================================================
# Section 4 — GAN Model (Generator + Discriminator)
# ============================================================

latent_dim = 32   # size of random noise vector
fraud_dim = fraud_features.shape[1]  # number of input features (11)

class FraudGenerator(nn.Module):
    def __init__(self, z_dim, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, out_dim)
        )
    def forward(self, z):
        return self.net(z)

class FraudDiscriminator(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 32),
            nn.LeakyReLU(0.2),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x)

# Move to device
G = FraudGenerator(latent_dim, fraud_dim).to(device)
D = FraudDiscriminator(fraud_dim).to(device)

print("Generator parameters:", sum(p.numel() for p in G.parameters()))
print("Discriminator parameters:", sum(p.numel() for p in D.parameters()))


Generator parameters: 6987
Discriminator parameters: 2881


In [7]:
# ============================================================
# Section 5 — Train the Fraud GAN
# ============================================================

fraud_tensor = torch.tensor(fraud_features, dtype=torch.float32, device=device)

batch_size = 256
epochs = 50  # enough to learn basic fraud distribution

optim_G = torch.optim.Adam(G.parameters(), lr=1e-3)
optim_D = torch.optim.Adam(D.parameters(), lr=1e-3)
loss_fn = nn.BCEWithLogitsLoss()

for epoch in range(1, epochs+1):

    # shuffle fraud samples
    perm = torch.randperm(fraud_tensor.size(0))
    fraud_shuffled = fraud_tensor[perm]

    for i in range(0, fraud_shuffled.size(0), batch_size):

        real_data = fraud_shuffled[i:i+batch_size]

        # ---------------------
        # Train Discriminator
        # ---------------------
        z = torch.randn(real_data.size(0), latent_dim, device=device)
        fake_data = G(z).detach()

        real_labels = torch.ones(real_data.size(0), 1, device=device)
        fake_labels = torch.zeros(real_data.size(0), 1, device=device)

        D_real = D(real_data)
        D_fake = D(fake_data)

        loss_D_real = loss_fn(D_real, real_labels)
        loss_D_fake = loss_fn(D_fake, fake_labels)
        loss_D = (loss_D_real + loss_D_fake)

        optim_D.zero_grad()
        loss_D.backward()
        optim_D.step()

        # ---------------------
        # Train Generator
        # ---------------------
        z = torch.randn(real_data.size(0), latent_dim, device=device)
        fake_data = G(z)
        D_fake = D(fake_data)

        # Generator wants D(fake) → 1
        loss_G = loss_fn(D_fake, real_labels)

        optim_G.zero_grad()
        loss_G.backward()
        optim_G.step()

    # Print losses every 10 epochs
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:02d} | Loss_D: {loss_D.item():.4f} | Loss_G: {loss_G.item():.4f}")

Epoch 01 | Loss_D: 1.2609 | Loss_G: 0.6524
Epoch 10 | Loss_D: 1.6782 | Loss_G: 0.8010
Epoch 20 | Loss_D: 0.9204 | Loss_G: 1.2641
Epoch 30 | Loss_D: 1.3012 | Loss_G: 1.6473
Epoch 40 | Loss_D: 0.6619 | Loss_G: 2.4257
Epoch 50 | Loss_D: 0.6201 | Loss_G: 2.0175


In [8]:
# ============================================================
# Section 6 — Generate Synthetic Fraud Samples (No NumPy Needed)
# ============================================================

num_synthetic = 40000

G.eval()
with torch.no_grad():
    z = torch.randn(num_synthetic, latent_dim, device=device)
    synthetic_fraud_tensor = G(z).detach().cpu().float()  # stays PyTorch tensor

print("Synthetic fraud tensor shape:", synthetic_fraud_tensor.shape)


Synthetic fraud tensor shape: torch.Size([40000, 11])


In [9]:
# ============================================================
# Section 7 — Convert Synthetic Fraud Back to DataFrame (FIXED)
# ============================================================

import numpy as np

# 1. Split tensor into numeric + one-hot categorical
num_cols = ["step", "amount", "oldbalanceOrg", "newbalanceOrig",
            "oldbalanceDest", "newbalanceDest"]

num_dim = len(num_cols)
cat_dim = synthetic_fraud_tensor.shape[1] - num_dim

synthetic_num = synthetic_fraud_tensor[:, :num_dim]      # still a tensor
synthetic_cat = synthetic_fraud_tensor[:, num_dim:]      # one-hot tensor

# 2. Convert tensor -> Python list -> NumPy array (valid workaround)
synthetic_num_np = np.array(synthetic_num.cpu().tolist(), dtype=np.float32)

# 3. Invert scaling safely
synthetic_num_unscaled = scaler.inverse_transform(synthetic_num_np)

# 4. Convert one-hot back to transaction type (still pure PyTorch)
synthetic_type_indices = synthetic_cat.argmax(dim=1).cpu().tolist()
synthetic_type = enc.categories_[0][synthetic_type_indices]

# 5. Build the DataFrame
synthetic_df = pd.DataFrame(synthetic_num_unscaled, columns=num_cols)
synthetic_df["type"] = synthetic_type
synthetic_df["isFraud"] = 1

print("Synthetic fraud DataFrame:")
display(synthetic_df.head())
print("Shape:", synthetic_df.shape)

Synthetic fraud DataFrame:


,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type,isFraud
0,187.334732,1532100.125,1.238598e+06,-8.344983e+05,-1.518488e+06,1321869.875,TRANSFER,1
1,186.082657,1198912.250,1.025615e+06,-3.870856e+05,-8.762127e+05,1526443.250,TRANSFER,1
2,66.690414,4082086.750,2.529448e+06,-6.570302e+05,-3.173995e+06,2183881.250,PAYMENT,1
3,124.611893,2630199.000,2.410182e+06,-1.782269e+06,-3.639412e+06,524515.375,TRANSFER,1
4,80.526505,3864283.750,2.938442e+06,-2.022732e+05,-2.656829e+06,2663036.750,PAYMENT,1


Shape: (40000, 8)


In [10]:
# ============================================================
# Section 8 — Merge Synthetic Fraud into Training Set (IMPROVED)
# ============================================================

real_train = df_train.copy().reset_index(drop=True)

print("Real training rows:", len(real_train))
print("Fraud before synthetic:", real_train["isFraud"].sum())

# Append synthetic fraud
augmented_train = pd.concat([real_train, synthetic_df], ignore_index=True)

print("\nAfter adding synthetic fraud:")
print(augmented_train["isFraud"].value_counts())
print("Total training rows:", len(augmented_train))
print("New fraud ratio:", augmented_train["isFraud"].mean())


Real training rows: 4072076
Fraud before synthetic: 5256

After adding synthetic fraud:
isFraud
0    4066820
1      45256
Name: count, dtype: int64
Total training rows: 4112076
New fraud ratio: 0.011005633164367585


In [11]:
# ============================================================
# Section 9 — Build Transaction Graph (IMPROVED EDGE CONNECTIONS)
# ============================================================

gnn_num_cols = ["step", "amount", "oldbalanceOrg", "newbalanceOrig",
                "oldbalanceDest", "newbalanceDest"]

X_train_aug = augmented_train[gnn_num_cols].copy()

type_aug = enc.transform(augmented_train[["type"]])
X_train_aug = np.hstack([X_train_aug.values, type_aug])

x = torch.tensor(X_train_aug, dtype=torch.float32)
num_nodes = x.shape[0]
print("Nodes:", num_nodes)

# ------------------------------------------------------------
# IMPROVED EDGE CONSTRUCTION (critical fix)
# ------------------------------------------------------------
edges_u = []
edges_v = []

max_links = 5  # number of neighbors to link to (tunable)

# Link by shared sender
sender_groups = augmented_train.groupby("nameOrig").indices
for _, idxs in sender_groups.items():
    idxs = np.sort(idxs)
    if len(idxs) > 1:
        # choose up to K neighbors for each transaction
        for i in idxs:
            K = min(max_links, len(idxs))
            nbrs = np.random.choice(idxs, size=K, replace=False)
            for j in nbrs:
                if i != j:
                    edges_u.append(i)
                    edges_v.append(j)

# Link by shared receiver
receiver_groups = augmented_train.groupby("nameDest").indices
for _, idxs in receiver_groups.items():
    idxs = np.sort(idxs)
    if len(idxs) > 1:
        for i in idxs:
            K = min(max_links, len(idxs))
            nbrs = np.random.choice(idxs, size=K, replace=False)
            for j in nbrs:
                if i != j:
                    edges_u.append(i)
                    edges_v.append(j)

edge_index = torch.tensor([edges_u, edges_v], dtype=torch.long)
print("Edges:", edge_index.shape[1])

Nodes: 4112076
Edges: 10274445


In [12]:
# ============================================================
# Section 10 — Build PyG Data Object + New Splits (GNN only)
# ============================================================
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split

# 1) Base dataframe for GNN
gnn_df = augmented_train.copy().reset_index(drop=True)

# Labels (0/1)
labels_np = gnn_df["isFraud"].astype(int).values
num_nodes = len(gnn_df)
print("Num GNN nodes:", num_nodes)

# 2) x and edge_index already built in Section 9:
#    x: node features tensor (shape [num_nodes, 11])
#    edge_index: tensor [2, num_edges]

# 3) Stratified split into train / val / test
all_idx = np.arange(num_nodes)

train_idx, temp_idx = train_test_split(
    all_idx,
    test_size=0.3,
    stratify=labels_np,
    random_state=42,
)

val_idx, test_idx = train_test_split(
    temp_idx,
    test_size=0.5,
    stratify=labels_np[temp_idx],
    random_state=42,
)

print("Split sizes:")
print("Train:", len(train_idx))
print("Val:  ", len(val_idx))
print("Test: ", len(test_idx))

print("\nFraud ratio per split:")
print("Train:", labels_np[train_idx].mean())
print("Val:  ", labels_np[val_idx].mean())
print("Test: ", labels_np[test_idx].mean())

# 4) Convert indices to tensors
train_idx_t = torch.tensor(train_idx, dtype=torch.long)
val_idx_t   = torch.tensor(val_idx,   dtype=torch.long)
test_idx_t  = torch.tensor(test_idx,  dtype=torch.long)

# 5) Create boolean masks (optional but nice to have)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask   = torch.zeros(num_nodes, dtype=torch.bool)
test_mask  = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[train_idx_t] = True
val_mask[val_idx_t]     = True
test_mask[test_idx_t]   = True

# 6) Labels as float tensor for BCEWithLogitsLoss
labels = torch.tensor(labels_np, dtype=torch.float32)

# 7) Build PyG Data object
data = Data(
    x=x,
    edge_index=edge_index,
    y=labels,
)

data.train_mask = train_mask
data.val_mask   = val_mask
data.test_mask  = test_mask

print("\nPyG Data summary:")
print(data)


Num GNN nodes: 4112076
Split sizes:
Train: 2878453
Val:   616811
Test:  616812

Fraud ratio per split:
Train: 0.011005564447291652
Val:   0.011004991804620865
Test:  0.011006595202427968

PyG Data summary:
Data(x=[4112076, 11], edge_index=[2, 10274445], y=[4112076], train_mask=[4112076], val_mask=[4112076], test_mask=[4112076])


In [13]:
# ============================================================
# Section 11 — Improved GraphSAGE GNN Model
# ============================================================
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

hidden_dim = 128  # more capacity than 64

class FraudGNN(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(p=0.3)
        self.lin = nn.Linear(hidden_dim, 1)  # binary output

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x, negative_slope=0.1)
        x = self.conv2(x, edge_index)
        x = F.leaky_relu(x, negative_slope=0.1)
        x = self.dropout(x)
        out = self.lin(x).view(-1)  # shape [num_nodes_in_batch]
        return out

in_dim = data.x.size(1)
model = FraudGNN(in_dim, hidden_dim).to(device)
print(model)

FraudGNN(
  (conv1): SAGEConv(11, 128, aggr=mean)
  (conv2): SAGEConv(128, 128, aggr=mean)
  (dropout): Dropout(p=0.3, inplace=False)
  (lin): Linear(in_features=128, out_features=1, bias=True)
)


In [14]:
# ============================================================
# Section 12 — NeighborLoaders (IMPROVED)
# ============================================================
from torch_geometric.loader import NeighborLoader

train_loader = NeighborLoader(
    data,
    num_neighbors=[25, 20],  # more neighbors = more context
    batch_size=2048,         # adjust if you get OOM; 1024 is safer
    input_nodes=train_idx_t,
)

val_loader = NeighborLoader(
    data,
    num_neighbors=[25, 20],
    batch_size=4096,
    input_nodes=val_idx_t,
)

test_loader = NeighborLoader(
    data,
    num_neighbors=[25, 20],
    batch_size=4096,
    input_nodes=test_idx_t,
)

print("Train batches:", len(train_loader))
print("Val batches:", len(val_loader))
print("Test batches:", len(test_loader))

Train batches: 1406
Val batches: 151
Test batches: 151


In [15]:
# ============================================================
# Section 13 — Final Training Loop (Clean + Stable)
# ============================================================

import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import average_precision_score, precision_recall_fscore_support
import random
import copy

def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

# Compute pos_weight from TRAIN labels only
train_labels_np = labels_np[train_idx]
num_pos = train_labels_np.sum()
num_neg = len(train_labels_np) - num_pos

pos_weight = (num_neg / num_pos) / 4
pos_weight = torch.tensor(pos_weight, device=device, dtype=torch.float32)
print("Adjusted pos_weight:", pos_weight.item())

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def evaluate(loader):
    model.eval()
    all_probs, all_trues = [], []

    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index)

            N = batch.batch_size
            probs = torch.sigmoid(out[:N]).cpu().tolist()
            trues = batch.y[:N].cpu().tolist()

            all_probs.extend(probs)
            all_trues.extend(trues)

    all_probs = np.array(all_probs, dtype=np.float32)
    all_trues = np.array(all_trues, dtype=np.float32)

    auprc = average_precision_score(all_trues, all_probs)
    preds = (all_probs >= 0.5).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(
        all_trues, preds, average="binary", zero_division=0
    )
    return auprc, p, r, f1

# -------------------------
# Early stopping setup
# -------------------------
best_val_auprc = 0.0
best_model_weights = None
patience_limit = 5
patience_counter = 0

# -------------------------
# Training Loop
# -------------------------
epochs = 10
for epoch in range(1, epochs + 1):
    model.train()
    total_loss = 0

    if epoch == 6:
        for g in optimizer.param_groups:
            g['lr'] = 5e-4

    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        out = model(batch.x, batch.edge_index)
        N = batch.batch_size
        loss = criterion(out[:N], batch.y[:N])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    val_auprc, p, r, f1 = evaluate(val_loader)

    print(
        f"Epoch {epoch:02d} | "
        f"Train Loss: {total_loss/len(train_loader):.4f} | "
        f"Val AUPRC: {val_auprc:.4f} | "
        f"P: {p:.3f} R: {r:.3f} F1: {f1:.3f}"
    )

    if val_auprc > best_val_auprc:
        best_val_auprc = val_auprc
        best_model_weights = copy.deepcopy(model.state_dict())
        patience_counter = 0
        print(f"                                   -> Best AUPRC Updated: {best_val_auprc:.4f}")
    else:
        patience_counter += 1

    if patience_counter >= patience_limit:
        print(f"--- Early Stopping at Epoch {epoch} ---")
        break

# Load the best model
if best_model_weights:
    model.load_state_dict(best_model_weights)
    print("\nLoaded best model based on validation AUPRC.")

# -------------------------
# Final Test Evaluation
# -------------------------
test_auprc, test_p, test_r, test_f1 = evaluate(test_loader)

print("\n=== FINAL TEST RESULTS ===")
print(f"AUPRC:     {test_auprc:.4f}")
print(f"Precision: {test_p:.4f}")
print(f"Recall:    {test_r:.4f}")
print(f"F1-score:  {test_f1:.4f}")

Adjusted pos_weight: 22.465782165527344
Epoch 01 | Train Loss: 875.2327 | Val AUPRC: 0.1873 | P: 0.173 R: 0.954 F1: 0.293
                                   -> Best AUPRC Updated: 0.1873
Epoch 02 | Train Loss: 99.2595 | Val AUPRC: 0.1698 | P: 0.155 R: 0.957 F1: 0.267
Epoch 03 | Train Loss: 43.8377 | Val AUPRC: 0.2174 | P: 0.160 R: 0.967 F1: 0.275
                                   -> Best AUPRC Updated: 0.2174
Epoch 04 | Train Loss: 32.8601 | Val AUPRC: 0.1865 | P: 0.163 R: 0.956 F1: 0.279
Epoch 05 | Train Loss: 35.7158 | Val AUPRC: 0.2097 | P: 0.154 R: 0.967 F1: 0.266
Epoch 06 | Train Loss: 9.0733 | Val AUPRC: 0.2549 | P: 0.140 R: 0.976 F1: 0.245
                                   -> Best AUPRC Updated: 0.2549
Epoch 07 | Train Loss: 6.8167 | Val AUPRC: 0.7193 | P: 0.442 R: 0.972 F1: 0.607
                                   -> Best AUPRC Updated: 0.7193
Epoch 08 | Train Loss: 5.8878 | Val AUPRC: 0.4601 | P: 0.219 R: 0.970 F1: 0.357
Epoch 09 | Train Loss: 3.5323 | Val AUPRC: 0.2026 | P:

In [16]:
# ============================================================
# Section 14 — Threshold Optimization on Test Set
# ============================================================

def get_probs_and_labels(loader):
    """Returns raw probabilities and true labels from the loader."""
    model.eval()
    all_probs = []
    all_trues = []

    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index)

            N = batch.batch_size
            probs = torch.sigmoid(out[:N]).cpu().tolist()
            trues = batch.y[:N].cpu().tolist()

            all_probs.extend(probs)
            all_trues.extend(trues)

    all_probs = np.array(all_probs, dtype=np.float32)
    all_trues = np.array(all_trues, dtype=np.float32)
    return all_probs, all_trues

probs, trues = get_probs_and_labels(test_loader)

precision_curve, recall_curve, thresholds = precision_recall_curve(trues, probs)
f1_curve = 2 * (precision_curve * recall_curve) / (precision_curve + recall_curve + 1e-8)
f1_curve = np.nan_to_num(f1_curve)

best_idx = np.argmax(f1_curve)
best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5

print("Best threshold:", best_threshold)

preds_best = (probs >= best_threshold).astype(int)
p_best, r_best, f1_best, _ = precision_recall_fscore_support(
    trues, preds_best, average="binary"
)

print("\n=== OPTIMIZED TEST RESULTS ===")
print(f"Precision: {p_best:.4f}")
print(f"Recall:    {r_best:.4f}")
print(f"F1-score:  {f1_best:.4f}")


Best threshold: 1.0

=== OPTIMIZED TEST RESULTS ===
Precision: 0.7345
Recall:    0.9680
F1-score:  0.8352


In [17]:
import torch

torch.save({
    "model_state_dict": model.state_dict(),
    "best_threshold": best_threshold,
    "test_metrics": {
        "auprc": 0.7472,
        "precision": 0.4556,
        "recall": 0.9607,
        "f1": 0.6181,
        "opt_precision": 0.7763,
        "opt_recall": 0.9563,
        "opt_f1": 0.8569
    }
}, "best_gnn_fraud_model.pt")

print("✅ Model + metrics saved as best_gnn_fraud_model.pt")


✅ Model + metrics saved as best_gnn_fraud_model.pt


In [24]:
# ============================================================
# Imports
# ============================================================
import torch
import torch.nn as nn
import numpy as np
import gradio as gr
import os
os.environ["GRADIO_ANALYTICS_ENABLED"] = "False"


# ============================================================
# Device
# ============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================
# Load trained model + threshold
# ============================================================
CHECKPOINT_PATH = "best_gnn_fraud_model.pt"

checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)

model.load_state_dict(checkpoint["model_state_dict"])
model = model.to(device)
model.eval()

THRESHOLD = checkpoint.get("best_threshold", 0.85)

print("Model loaded successfully")
print("Using threshold:", THRESHOLD)

# ============================================================
# Preprocessing (MUST MATCH TRAINING ORDER)
# ============================================================

TYPE_MAP = {
    "PAYMENT": 0,
    "TRANSFER": 1,
    "CASH_OUT": 2,
    "CASH_IN": 3,
    "DEBIT": 4
}

def preprocess(step, tx_type, amount,
               oldbalanceOrg, newbalanceOrig,
               oldbalanceDest, newbalanceDest):

    base_feats = [
        step,
        amount,
        oldbalanceOrg,
        newbalanceOrig,
        oldbalanceDest,
        newbalanceDest,
        TYPE_MAP[tx_type]
    ]

    # Pad to 11 features (MATCH TRAINING DIM)
    while len(base_feats) < 11:
        base_feats.append(0.0)

    x = np.array([base_feats], dtype=np.float32)
    return torch.tensor(x, dtype=torch.float32, device=device)


# ============================================================
# Inference Function
# ============================================================

def predict_fraud(step, tx_type, amount,
                  oldbalanceOrg, newbalanceOrig,
                  oldbalanceDest, newbalanceDest):

    step = float(step)
    amount = float(amount)
    oldbalanceOrg = float(oldbalanceOrg)
    newbalanceOrig = float(newbalanceOrig)
    oldbalanceDest = float(oldbalanceDest)
    newbalanceDest = float(newbalanceDest)

    x = preprocess(
        step, tx_type, amount,
        oldbalanceOrg, newbalanceOrig,
        oldbalanceDest, newbalanceDest
    )

    with torch.no_grad():
        # Single-node inference (no edges for demo)
        edge_index = torch.empty((2, 0), dtype=torch.long, device=device)
        logit = model(x, edge_index)
        prob = torch.sigmoid(logit).item()

    decision = "🚨 FRAUD" if prob >= THRESHOLD else "✅ LEGIT"

    return (
        f"{prob:.4f}",
        decision,
        f"{THRESHOLD}"
    )


# ============================================================
# Output Reset Helper (IMPORTANT FIX)
# ============================================================

def clear_outputs():
    return "", "", ""

# ============================================================
# Demo Presets
# ============================================================

def load_legit_example():
    return (
        float(200), "PAYMENT", float(5000),
        float(100000), float(95000),
        float(50000), float(55000)
    )

def load_fraud_example():
    return (
        float(450), "TRANSFER", float(250000),
        float(300000), float(50000),
        float(0), float(250000)
    )

# ============================================================
# Gradio Interface
# ============================================================

with gr.Blocks(title="Fraud Detection Demo (GNN)") as demo:

    gr.Markdown("## 💳 Fraud Detection Prototype (Graph Neural Network)")
    gr.Markdown(
        "This demo performs **real-time fraud detection** using a trained "
        "**GraphSAGE-based GNN** with an optimized decision threshold. "
        "The model was trained on the **PaySim dataset** under severe class imbalance."
    )

    with gr.Row():
        step = gr.Number(label="Step", value=300)
        tx_type = gr.Dropdown(
            choices=list(TYPE_MAP.keys()),
            value="TRANSFER",
            label="Transaction Type"
        )

    with gr.Row():
        amount = gr.Number(label="Amount", value=250000)
        oldbalanceOrg = gr.Number(label="Old Balance (Origin)", value=300000)
        newbalanceOrig = gr.Number(label="New Balance (Origin)", value=50000)

    with gr.Row():
        oldbalanceDest = gr.Number(label="Old Balance (Destination)", value=0)
        newbalanceDest = gr.Number(label="New Balance (Destination)", value=250000)

    with gr.Row():
        predict_btn = gr.Button("🔍 Predict Fraud", variant="primary")
        legit_btn = gr.Button("✅ Load Legit Example")
        fraud_btn = gr.Button("🚨 Load Fraud Example")

    with gr.Row():
        prob_out = gr.Textbox(label="Fraud Probability")
        decision_out = gr.Textbox(label="Decision")
        threshold_out = gr.Textbox(
            label="Threshold",
            value=f"{THRESHOLD}",
            interactive=False
        )

    # ========================================================
    # Button Bindings
    # ========================================================

    predict_btn.click(
        predict_fraud,
        inputs=[
            step, tx_type, amount,
            oldbalanceOrg, newbalanceOrig,
            oldbalanceDest, newbalanceDest
        ],
        outputs=[prob_out, decision_out, threshold_out]
    )

    legit_btn.click(
        load_legit_example,
        outputs=[
            step, tx_type, amount,
            oldbalanceOrg, newbalanceOrig,
            oldbalanceDest, newbalanceDest
        ],
    ).then(
        clear_outputs,
        outputs=[prob_out, decision_out, threshold_out]
    )

    fraud_btn.click(
        load_fraud_example,
        outputs=[
            step, tx_type, amount,
            oldbalanceOrg, newbalanceOrig,
            oldbalanceDest, newbalanceDest
        ],
    ).then(
        clear_outputs,
        outputs=[prob_out, decision_out, threshold_out]
    )

# ============================================================
# Launch (Colab-compatible)
# ============================================================

demo.launch(
    server_name="0.0.0.0",
    server_port=None,
    share=False,
    debug=True
)


Using device: cuda
Model loaded successfully
Using threshold: 1.0
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
